In [62]:
##Import Key Libraries for use
import datetime as dt
import math
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error

In [63]:
#Import CSV
covid19Dataframe = pd.read_csv("https://raw.githubusercontent.com/WhipSnake23/Python-Class-Project/main/Data/owid-covid-data.csv")
#Filter Dataset to Canada
covid19Dataframe = covid19Dataframe[covid19Dataframe.iso_code.str.contains("CAN")]
#Remove Smoothed Columns from dataframe

covid19Dataframe = covid19Dataframe.drop(columns=['new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million','new_tests_smoothed','new_tests_smoothed_per_thousand','new_vaccinations_smoothed','new_vaccinations_smoothed_per_million'])
#Reset Index
covid19Dataframe.reset_index(drop=True, inplace=True)
#covid19Dataframe.dropna(inplace=True)
covid19Dataframe.dropna(subset=['total_deaths'],inplace=True)

In [64]:
#Set x and y
#print(covid19Dataframe.head)

columns = ["total_cases", "new_cases", "median_age", "gdp_per_capita"]
target = "total_deaths"

Features=covid19Dataframe[columns]
target=covid19Dataframe[target]

print(Features)
print(target)

     total_cases  new_cases  median_age  gdp_per_capita
43          77.0       13.0        41.4       44017.591
44          79.0        2.0        41.4       44017.591
45         108.0       29.0        41.4       44017.591
46         117.0        9.0        41.4       44017.591
47         193.0       76.0        41.4       44017.591
..           ...        ...         ...             ...
388     839155.0     2561.0        41.4       44017.591
389     842590.0     3435.0        41.4       44017.591
390     845673.0     3083.0        41.4       44017.591
391     848330.0     2657.0        41.4       44017.591
392     850737.0     2407.0        41.4       44017.591

[350 rows x 4 columns]
43         1.0
44         1.0
45         1.0
46         1.0
47         1.0
        ...   
388    21439.0
389    21509.0
390    21581.0
391    21631.0
392    21675.0
Name: total_deaths, Length: 350, dtype: float64


In [65]:
X_train, X_test, y_train, y_test = train_test_split(Features, target, test_size=0.25, random_state=1)

In [66]:
def printMetrics(test, predictions):
    print(f"Score: {explained_variance_score(test, predictions):.2f}")
    print(f"MAE: {mean_absolute_error(test, predictions):.2f}")
    print(f"RMSE: {math.sqrt(mean_squared_error(test, predictions)):.2f}")
    print(f"r2: {r2_score(test, predictions):.2f}")

reg = ElasticNet()
reg.fit(X_train, y_train)
print(reg.score(X_test, y_test))
predictions = reg.predict(X_test)
printMetrics(y_test, predictions)

0.8891313496981199
Score: 0.89
MAE: 1548.75
RMSE: 1921.68
r2: 0.89
